In [ ]:
%%bash
set -euxo pipefail
export DEBIAN_FRONTEND=noninteractive

echo "[STEP 1/6] base packages $(date -u)"
apt-get update
apt-get install -y --no-install-recommends \
  ca-certificates git curl unzip zip openjdk-17-jdk-headless

echo "[DONE] step 1/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 2/6] android cmdline-tools $(date -u)"
mkdir -p "$ANDROID_SDK_ROOT/cmdline-tools"
if [ ! -x "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" ]; then
  curl -fL --retry 5 --retry-delay 5 \
    https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip \
    -o /tmp/android-cmdline-tools.zip
  unzip -q /tmp/android-cmdline-tools.zip -d "$ANDROID_SDK_ROOT/cmdline-tools"
  rm -rf "$ANDROID_SDK_ROOT/cmdline-tools/latest"
  mv "$ANDROID_SDK_ROOT/cmdline-tools/cmdline-tools" "$ANDROID_SDK_ROOT/cmdline-tools/latest"
fi

echo "[DONE] step 2/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 3/6] accept sdk licenses $(date -u)"
set +o pipefail
timeout 900 bash -lc 'yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/android-sdk --licenses --verbose' \
  > /tmp/sdk-licenses.log 2>&1
licenses_rc=$?
set -o pipefail
tail -n 120 /tmp/sdk-licenses.log || true
if [ "$licenses_rc" -eq 124 ]; then
  echo "sdkmanager --licenses timeout (15 min)"
  exit 124
fi
if [ "$licenses_rc" -ne 0 ]; then
  echo "sdkmanager --licenses failed: $licenses_rc"
  exit "$licenses_rc"
fi

echo "[DONE] step 3/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 4/6] install sdk packages $(date -u)"
set +o pipefail
yes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT" \
  "platform-tools" "platforms;android-34" "build-tools;34.0.0"
sdk_rc=${PIPESTATUS[1]}
set -o pipefail
if [ "$sdk_rc" -ne 0 ]; then
  echo "sdkmanager packages failed: $sdk_rc"
  exit "$sdk_rc"
fi

echo "[DONE] step 4/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail

echo "[STEP 5/6] write env $(date -u)"
cat > /content/tgwebmobile_env.sh <<'ENVEOF'
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export ANDROID_SDK_ROOT=/content/android-sdk
export ANDROID_HOME=$ANDROID_SDK_ROOT
export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
ENVEOF

echo "[DONE] step 5/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[STEP 6/6] clone project + prepare local.properties $(date -u)"
cd /content
if [ ! -d tgwebmobile ]; then
  git clone https://github.com/Perdonus/tgwebmobile.git
fi

cd /content/tgwebmobile
git fetch origin main
git reset --hard origin/main

cat > local.properties <<EOF2
sdk.dir=/content/android-sdk
EOF2

# Optional files (upload manually to /content before this step):
# /content/google-services.json, /content/keystore.jks, /content/keystore.properties
if [ -f /content/google-services.json ]; then cp /content/google-services.json app/google-services.json; fi
if [ -f /content/keystore.jks ]; then cp /content/keystore.jks app/keystore.jks; fi
if [ -f /content/keystore.properties ]; then cp /content/keystore.properties keystore.properties; fi

chmod +x gradlew
echo "[DONE] step 6/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

cd /content
if [ ! -d tgwebmobile ]; then
  env -u HTTP_PROXY -u HTTPS_PROXY -u ALL_PROXY -u http_proxy -u https_proxy -u all_proxy \
    git clone -b main https://github.com/Perdonus/tgwebmobile.git
fi

cd /content/tgwebmobile
env -u HTTP_PROXY -u HTTPS_PROXY -u ALL_PROXY -u http_proxy -u https_proxy -u all_proxy git fetch origin main
env -u HTTP_PROXY -u HTTPS_PROXY -u ALL_PROXY -u http_proxy -u https_proxy -u all_proxy git reset --hard origin/main
git --no-pager log --oneline -n 5


In [ ]:
%%bash
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[BUILD] start $(date -u)"
cd /content/tgwebmobile
./gradlew --no-daemon :app:assembleDebug
echo "[BUILD] done $(date -u)"
